In [ ]:
import shutil,os
if not os.path.exists("/content/mrcnn"):
    !git clone https://github.com/matterport/Mask_RCNN.git
    shutil.move("/Mask_RCNN/mrcnn","/")
    !pip install -r "/content/Mask_RCNN/requirements.txt"
    shutil.copy("/content/drive/MyDrive/ISIC 2018 Dataset/model.py","/content/mrcnn")
    shutil.copy("/content/drive/MyDrive/ISIC 2018 Dataset/saving.py","/tensorflow-1.15.2/python3.7/keras/engine")
    shutil.rmtree("/content/Mask_RCNN")

# **Imports and Path Initializations**

In [ ]:
from imgaug import augmenters as iaa
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import visualize
from mrcnn import utils
from imutils import paths
import numpy as np
import argparse
import imutils
import random
import cv2
import os
from PIL import Image
import sys

# initialize the dataset path, images path, and masks file path
DATASET_PATH = os.path.abspath("/content/drive/My Drive/ISIC 2018 Dataset")

# initialize the class names dictionary
CLASS_NAMES = {1: "lesion"}

# initialize the name of the directory where logs and output model
# snapshots will be stored
LOGS_AND_MODEL_DIR = "drive/My Drive/ISIC 2018 Dataset/lesions_logs"

In [ ]:
class LesionConfig(Config):
	# give the configuration a recognizable name
	NAME = "lesion"

	# set the number of GPUs to use training along with the number of
	# images per GPU (which may have to be tuned depending on how
	# much memory your GPU has)
	GPU_COUNT = 1
	IMAGES_PER_GPU = 6

	# set the number of steps per training epoch and validation cycle
	STEPS_PER_EPOCH = train_size // (IMAGES_PER_GPU * GPU_COUNT)
	VALIDATION_STEPS = val_size // (IMAGES_PER_GPU * GPU_COUNT)

	# number of classes (+1 for the background)
	NUM_CLASSES = len(CLASS_NAMES) + 1

	IMAGE_MIN_DIM = 512
	IMAGE_MAX_DIM = 512
	RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)

class LesionBoundaryInferenceConfig(LesionConfig):
	# set the number of GPUs and images per GPU (which may be
	# different values than the ones used for training)
	GPU_COUNT = 1
	IMAGES_PER_GPU = 8

	# set the minimum detection confidence (used to prune out false
	# positive detections)
	DETECTION_MIN_CONFIDENCE = 0.9

class LesionDataset(utils.Dataset):
	def __init__(self, imagePaths, masksPath, classNames, width=512):
		# call the parent constructor
		super().__init__(self)

		# store the image paths and class names along with the width
		# we'll resize images to
		self.imagePaths = imagePaths
		self.masksPath = masksPath
		self.classNames = classNames
		self.width = width

	def load_lesions(self, idxs):
		# loop over all class names and add each to the 'lesion'
		# dataset
		for (classID, label) in self.classNames.items():
			self.add_class("lesion", classID, label)
   
    # loop over the image path indexes  
		for i in range(idxs):
    # extract the image filename to serve as the unique
    # image ID
			imagePath = self.imagePaths[i]
			filename = imagePath.split(os.path.sep)[-1]
      # add the image to the dataset
			self.add_image("lesion", image_id=filename,path=imagePath)
            
	def load_image(self, imageID):
		# grab the image path, load it, and convert it from BGR to
		# RGB color channel ordering
		p = self.image_info[imageID]["path"]
		image = cv2.imread(p)
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

		# resize the image, preserving the aspect ratio
		image = imutils.resize(image, width=self.width)

		# return the image
		return image

	def load_mask(self, imageID):
		# grab the image info and derive the full annotation path
		# file path
		info = self.image_info[imageID]
		filename = info["id"].split(".")[0]
		annotPath = os.path.sep.join([self.masksPath,
			"{}_segmentation.png".format(filename)])

		# load the annotation mask and resize it, *making sure* to
		# use nearest neighbor interpolation
		annotMask = cv2.imread(annotPath)
		annotMask = cv2.split(annotMask)[0]
		annotMask = imutils.resize(annotMask, width=self.width,
			inter=cv2.INTER_NEAREST)
		annotMask[annotMask > 0] = 1

		# determine the number of unique class labels in the mask
		classIDs = np.unique(annotMask)

		# the class ID with value '0' is actually the background
		# which we should ignore and remove from the unique set of
		# class identifiers
		classIDs = np.delete(classIDs, [0])

		# allocate memory for our [height, width, num_instances]
		# array where each "instance" effectively has its own
		# "channel" -- since there is only one lesion per image we
		# know the number of instances is equal to 1
		masks = np.zeros((annotMask.shape[0], annotMask.shape[1], 1),
			dtype="uint8")

		# loop over the class IDs
		for (i, classID) in enumerate(classIDs):
			# construct a mask for *only* the current label
			classMask = np.zeros(annotMask.shape, dtype="uint8")
			classMask[annotMask == classID] = 1

			# store the class mask in the masks array
			masks[:, :, i] = classMask

		# return the mask array and class IDs
		return (masks.astype("bool"), classIDs.astype("int32"))

# `TESTING`

In [ ]:
from cv2 import imshow
  # initialize the inference configuration
config = LesionBoundaryInferenceConfig()

		# initialize the Mask R-CNN model for inference
model = modellib.MaskRCNN(mode="inference", config=config,
	model_dir=LOGS_AND_MODEL_DIR)

		# load our trained Mask R-CNN
weights = model.find_last()
model.load_weights(weights, by_name=True)

In [ ]:
TEST_PATH = os.path.sep.join([DATASET_PATH,
	"Test"])
TEST_PATHS = sorted(list(paths.list_images(TEST_PATH)))
images=['']*config.BATCH_SIZE
passes = (len(TEST_PATHS)//config.BATCH_SIZE) +1
for k in range(0,passes):
	for i in range(0,config.BATCH_SIZE):
		if not (config.BATCH_SIZE*k + i>=len(TEST_PATHS)):
			image = cv2.imread(TEST_PATHS[config.BATCH_SIZE*k+i])
			image = imutils.resize(image, width=512)
			image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
			images[i]=image
	# perform a forward pass of the network to obtain the results
	result = model.detect(images, verbose=1)
	for j in range(0,len(result)):
		r=result[j]
	# loop over the predicted scores
		maxScore= 0
		for i in range(0, len(r["scores"])):
		# extract the bounding box information, class ID, label,
		# and predicted probability from the results
			if r["scores"][i]>maxScore:
				maxScore=r["scores"][i]
				(startY, startX, endY, endX) = r["rois"][i]
		for i in range(0, r["rois"].shape[0]):
			mask = r["masks"][:, :, i]

			image = images[j]
			image = visualize.draw_box(image, r["rois"][i],
			(1.0, 0.0, 0.0))
		image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
		# loop over the predicted scores and class labels
		for i in range(0, len(r["scores"])):
			# extract the bounding box information, class ID, label,
			# and predicted probability from the results
			(startY, startX, endY, end) = r["rois"][i]
			classID = r["class_ids"][i]
			label = CLASS_NAMES[classID]
			score = r["scores"][i]

			# draw the score on the image
			text = str(score)
			y = startY - 10 if startY - 10 > 10 else startY + 10
			cv2.putText(image, text, (startX, y),
			cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

		# resize the image so it more easily fits on our screen
		image = imutils.resize(image, width=256)

		# show the output image
		cv2_imshow(image)	 
	print(str(k*100//passes)+'%')

# **Lesion Segmentation**

In [ ]:
TEST_PATH = "/content/drive/MyDrive/ISIC 2019 Dataset/TrainingAugmented"
TEST_PATHS = [f for f in sorted(paths.list_images(TEST_PATH))]
len(TEST_PATHS)
images=['']*config.BATCH_SIZE
passes = (len(TEST_PATHS)//config.BATCH_SIZE) +1
for k in range(0,passes):
	for i in range(0,config.BATCH_SIZE):
		if not (config.BATCH_SIZE*k + i>=len(TEST_PATHS)):
			image = cv2.imread(TEST_PATHS[config.BATCH_SIZE*k+i])
			image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
			images[i]=image
	# perform a forward pass of the network to obtain the results
	result = model.detect(images, verbose=1)
	for j in range(0,len(result)):
		r=result[j]
	# loop over the predicted scores
		maxScore= 0
		for i in range(0, len(r["scores"])):
		# extract the bounding box information, class ID, label,
		# and predicted probability from the results
			if r["scores"][i]>maxScore:
				maxScore=r["scores"][i]
				(startY, startX, endY, endX) = r["rois"][i]
		if not config.BATCH_SIZE*k + j>=len(TEST_PATHS):
			if maxScore>0:
				imagePath = TEST_PATHS[config.BATCH_SIZE*k + j]
				imageName = os.path.basename(imagePath)
				im = Image.open(imagePath)
				width, height = im.size
				box = (startX, startY, endX, endY)
				crop = im.crop(box)
				crop.save('/content/drive/MyDrive/ISIC 2019 Dataset/ISIC_2019_Training_Input/'+imageName, 'JPEG')
			else:
				imagePath = TEST_PATHS[config.BATCH_SIZE*k + j]
				imageName = os.path.basename(imagePath)
				shutil.copy(imagePath,'/content/drive/MyDrive/ISIC 2019 Dataset/ISIC_2019_Training_Input/'+imageName)
	print(str(k*100//passes)+'%')